In [85]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle

In [86]:
features=pd.read_csv("./heart.csv")
features=shuffle(features)
features.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
296,63,0,0,124,197,0,1,136,1,0.0,1,0,2,0
163,38,1,2,138,175,0,1,173,0,0.0,2,4,2,1
263,63,0,0,108,269,0,1,169,1,1.8,1,2,2,0
182,61,0,0,130,330,0,0,169,0,0.0,2,0,2,0
69,62,0,0,124,209,0,1,163,0,0.0,2,0,2,1


In [87]:
labels=features.pop("output")

In [88]:
features=np.array(features)
lables=np.array(labels).reshape(-1,1)

In [89]:
features = ss().fit_transform(features)
features[0]

array([ 0.9521966 , -1.46841752, -0.93851463, -0.43541521, -0.95205107,
       -0.41763453,  0.89896224, -0.59678428,  1.43548113, -0.89686172,
       -0.64911323, -0.71442887, -0.51292188])

In [90]:
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)
print(features.shape)
print(labels.shape)

torch.Size([303, 13])
torch.Size([303])


In [91]:
model = torch.nn.Sequential(
    torch.nn.Linear(13,9),
    torch.nn.ReLU(),
    torch.nn.Linear(9,3),
    torch.nn.ReLU(),
    torch.nn.Linear(3,1),
    torch.nn.Sigmoid()
)

In [92]:
loss_fn = torch.nn.BCELoss()
opt = torch.optim.SGD(model.parameters(), lr=1e-3)

In [93]:
epochs = 10000

for epoch in range(epochs):
    preds = model(features)
    preds=preds.reshape(-1,1)
    loss = loss_fn(preds,labels)

    loss.backward()

    opt.step()
    opt.zero_grad()

    with torch.no_grad():
        if (epoch+1)%(epochs//10) == 0:
            right = 0
            for i in range(len(preds)):
                if preds[i][0] >= 0.5:
                    preds[i][0] = 1
                else:
                    preds[i][0] = 0

                if preds[i][0] == labels[i][0]:
                    right += 1

            print(f"Loss : {loss}")
            print(f"Accuracy : {round(right * 100/ len(preds), 2)}%")


ValueError: Using a target size (torch.Size([303])) that is different to the input size (torch.Size([303, 1])) is deprecated. Please ensure they have the same size.